In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pyvis.network import Network as net

In [2]:
g = nx.read_gpickle("objects/graph-A_3_0")

In [3]:
print(nx.info(g))

DiGraph named 'Graph from soil food web adjacency matrix' with 24 nodes and 94 edges


In [4]:
# Import trophic level as node attribute
trophic_levels = pd.read_csv("data/trophic_levels_1_0.csv", index_col=0, header=None, delimiter=";", names=["trophic_level"])
trophic_levels = trophic_levels.squeeze()

# Create dict from trophic level Series
troph_lev_dict = trophic_levels.to_dict()
troph_lev_dict

{'11 Plant-feeding nematode': 1,
 '31 Bacterivore nematode': 1,
 '45 Substrate-ingesting earthworm': 1,
 '23 Fungivore insect': 1,
 '24 Fungivore enchytraeid': 1,
 '21 Fungivore nematode': 1,
 '81 Omnivore nematode': 2,
 '13 Plant-feeding insect (springtail)': 1,
 '44 Substrate-inhabiting enchytraeid': 1,
 '51 Predating nematode (consuming nematodes)': 3,
 '72 Generalist mite': 3,
 '34 Bacterivore enchytraeid': 1,
 '82 Omnivore mite': 2,
 '12 Macrophytophage and panphytophage mite': 1,
 '36 Bacterivore amoebae': 1,
 '37 Bacterivore flagellates': 1,
 '#0 Plant autotrophs (roots)': 0,
 '41 "Passive lifestage, substrate-related nematode"': 1,
 '48 Primary (heterotrophic) producer bacteria': 0,
 '49 Primary (heterotrophic) producer fungi': 0,
 '22 Microphytophage mite (feeding on fungi)': 1,
 '62 Predatory mite (attacking arthropods)': 3,
 '52 Predatory mite (attacking nematodes)': 3,
 '92 Predatory mite (parasitizing mites and nematodes)': 3}

In [5]:
trophic_levels = list(troph_lev_dict.values())
colours = ["#40E0D0", "#DFFF00", "#FFBF00", "#DE3163"]
tlevel2color = {lev: colours[lev] for lev in trophic_levels}

In [6]:
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

# Create a PyVis graph G
G = net('800px', '1200px')
nodes = list(g.nodes())
for node in g.nodes():
    G.add_node(node, 
               label='\n'.join(chunkstring(node,15)), 
               level=max(trophic_levels) - troph_lev_dict[node], 
               shape='box',
               color=tlevel2color[troph_lev_dict[node]])

In [7]:
# Add edges to PyVis graph G
for source, target in list(g.edges()):
    G.add_edge(source, target)

In [8]:
# Edges take on the color of the node they are coming from.
G.inherit_edge_colors(status=True)

In [9]:
options= """
var options = {
  "edges": {
    "arrows": {
      "middle": {
        "enabled": true,
        "scaleFactor": 0.85
      }
    },
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "layout": {
    "hierarchical": {
      "enabled": true,
      "nodeSpacing": 200
    }
  },
  "interaction": {
    "hover": true,
    "navigationButtons": true,
    "multiselect": true
  },
  "manipulation": {
    "enabled": true,
    "initiallyActive": true
  },
  "physics": {
    "enabled": true,
    "hierarchicalRepulsion": {
      "centralGravity": 0,
      "nodeDistance": 190
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  },
  "configure": {
    "enabled": true
  }
}
"""
# G.write_html('SFW_2.html')
G.conf = True
G.set_options(options)
# G.show_buttons(filter_=True)  # if we enable this after setting options there is a bug so we use conf=True and JS settings
G.write_html('SFW_2.html')